# Evaluate the effects of PS on P, TV, F, sEAdi etc

## Read-me
<!-- TODO ... Insert description ... -->

The code to consecutively:
<!-- TODO  Update code steps-->

## 1. Import code libraries

In [ ]:
# Standard code libraries
import os
import sys
import glob
from pathlib import Path

import scipy
from scipy import interpolate as interp
from scipy.optimize import curve_fit
import pandas as pd
import numpy as np
import numpy.matlib

import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import datetime
%matplotlib widget

In [ ]:
# Custom code libraries from the ReSurfEMG repository
# It uses the ReSurfEMG library version v0.2.1

import resurfemg.preprocessing.ecg_removal as ecg_rm
import resurfemg.preprocessing.envelope as evl
import resurfemg.preprocessing.filtering as filt
import resurfemg.postprocessing.features as feat

from resurfemg.postprocessing.baseline import moving_baseline, slopesum_baseline
from resurfemg.postprocessing.event_detection import (
    onoffpeak_baseline_crossing, onoffpeak_slope_extrapolation)
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader
from resurfemg.helper_functions.helper_functions import derivative

from resurfemg.config.config import Config
config = Config()

## Initiation of output folder for experiments

In [ ]:
# Input data path - The main directory where all data is loaded from
root_patient_data_directory = \
    config.get_directory('root_patient_data_directory')

# Output data - General path to dir for saving .csvs and plots
main_output_dir = os.path.join(config.get_directory('preprocessed'),
                    '2024-05_PS12_evaluation_EMG')

if not os.path.exists(main_output_dir):
    os.makedirs(main_output_dir)

patient_idx = 0

In [ ]:
# Load measurement_files of interest

ps12_file_list = os.path.abspath(
    os.path.join(root_patient_data_directory, '..', '0_CRF_exports'))
ps12_df = pd.read_csv(
    os.path.join(ps12_file_list, 'PS12_levels.csv'), delimiter=';')
ps12_it = 39

In [ ]:
# ps12_df

In [ ]:
ps12_it,ps12_df.loc[ps12_it]

## 2. Load the ventilator and sEMG data

2.a Select a patient

In [ ]:
# Select the patient of interest
# Expected data structure:
# - Patient_01
# -- Measurement_date_XXXX_XX_01
# --- 001_Baseline
# --- 002_PS_step_01
# --- 003_PS_step_02
# --- 004_PS_step_03
# --- 005_PS_step_04
# -- Measurement_date_XXXX_XX_03
# --- 001_Baseline
# --- 002_PS_step_01
# --- 003_PS_step_02
# --- 004_PS_step_03
# --- 005_PS_step_04
# -- Patient_02
# -- Measurement_date_XXXX_XX_01
# etc.

# NB Run this cell once per patient!

patient_folders = glob.glob(
        os.path.join(root_patient_data_directory, '**',''), 
        recursive=False)

patients = []
for folder in patient_folders:
    name = Path(folder).parts[-1]
    patients.append(name)

patients.sort()

btn_pt = widgets.Dropdown(  
    options=patients,
    value=ps12_df.loc[ps12_it, 'PT'],
    description='Select patient:',
    disabled=False,
)

date_idx = 0

display(btn_pt)

2.b Select a measurement date, or PS trial

In [ ]:
# Select the day (PS trial) of interest for the selected patient
# measurement_date ~ PEEP-trial
 
# NB Run this cell once per patient/PEEP trial combination

patient = btn_pt.value
patient_idx =btn_pt.index

measurement_folders = glob.glob(
    os.path.join(root_patient_data_directory, patient, '**',''),
    recursive=False)
measurement_dates = []

for folder in measurement_folders:
    name = Path(folder).parts[-1]
    measurement_dates.append(name)

measurement_dates.sort()

# Initialise the analysis: empty the output parameter list and start at the 
# baseline measurement (index 0)
di_data_list = []
para_data_list = []
Paw_data_list = []
Vvent_data_list =[]
ecg_data_list = []

ecg_data_all_dict = dict()
df_ecg_all_dict = dict()
channel_list = ['ecg', 'di', 'para']
columns_ecg_all = ['patient', 'measurement',
              'ecg_minmax', 'ecg_min', 'ecg_max', 'ecg_method']
for idx, channel  in enumerate(channel_list):
    ecg_data_all_dict[channel] = []
    df_ecg_all_dict[channel] = pd.DataFrame(ecg_data_all_dict[channel], columns=columns_ecg_all)

PS_step_idx = 0
plt.close('all')

# Set the default pipeline parameters
# Gating settings
gate_width_default = 0.10
gate_threshold_default = 0.30
gate_ECG_shift_default = -10
gate_twice = False

# RMS window
RMS_window_ms_default = 200

# Peak detection settings
time_shift_default = 0.5 - RMS_window_ms_default/1000/2
sEAdi_prominence_factor_default = 0.5
sEApara_prominence_factor_default = 0.5

date_str = str(datetime.strptime(ps12_df.loc[ps12_it, 'DATE'], '%d-%m-%Y').date())
[date.startswith(date_str) for date in measurement_dates]

iteration_date = measurement_dates[
[i for i, date in enumerate(measurement_dates) if date.startswith(date_str)][0]]

btn_measurement = widgets.Dropdown(
    options=measurement_dates,
    value=iteration_date,
    description='Select measurement date:',
    parasabled=False,
)
display(btn_measurement)

2.c Select a PS step

In [ ]:
# Select the PS step of interest for the selected patient/measurement_date

# NB Re-run this cell for each new PEEP trial, as it also empties output 
# parameter list (di_data_list)!


# Create output data folders
measurement_date = btn_measurement.value
date_idx = btn_measurement.index

output_path = os.path.join(main_output_dir, patient, measurement_date)
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Identify all PS step folders:
root_emg_directory = os.path.join(
    root_patient_data_directory, patient, measurement_date)

emg_pattern = os.path.join(root_emg_directory, '**/*.Poly5')
emg_and_vent_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
vent_files = []
plt.close('all')

for file in emg_and_vent_files:
    if 'Draeger' in file:
        vent_files.append(file)
    else:
        emg_files.append(file)

emg_files.sort()
vent_files.sort()

list_of_numbers_string = []

for i in range(len(emg_files)):
    list_of_numbers_string.append(Path(emg_files[i]).parts[-2])

# Select the PEEP step of interest. The selection menu initialises at the third 
# but last (index -4) recording. The PEEP steps are named after the folders 
# containing the data files (.poly5) of interest.

ps12_idx = 5-ps12_df.loc[ps12_it, 'END-MEASUREMENT']

if len(list_of_numbers_string) < 9:
    ps12_idx - 1


btn_PS_step = widgets.Dropdown(
    options=list_of_numbers_string,
    value= list_of_numbers_string[ps12_idx],
    description='Picked File:',
    disabled=False,
)
display(btn_PS_step)

In [ ]:
# Process selected option: the PEEP step of interest  
PS_step_chosen = btn_PS_step.value
PS_step_idx = int(btn_PS_step.index)

print("The selected measurement is:\n", 
      patient, ' - ', ps12_df.loc[ps12_it, 'PT'], '\n',
      measurement_date[:10], ' - ', ps12_df.loc[ps12_it, 'DATE'], '\n',
      PS_step_chosen, ' - ', 5-ps12_df.loc[ps12_it, 'END-MEASUREMENT']+1,
      '/', len(list_of_numbers_string),
      )

In [ ]:
# Process selected option: the PS step of interest  
emg_file_chosen = emg_files[PS_step_idx]
vent_file_chosen = vent_files[PS_step_idx]
print("The chosen files are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
fs_vent = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples
# # Reshufle the channels if necessary
# y_emg = np.array([data_emg_samples[2, :], 
#                    data_emg_samples[0, :], 
#                    data_emg_samples[1, :]])
y_vent = data_vent_samples

mb_percentile = 50
mb_window = 60*fs_vent
# V_vent_bs = moving_baseline(y_vent[2, :], fs_vent, 20*fs_vent, mb_percentile)

V_vent_bs = moving_baseline(y_vent[2, :],
    20*fs_vent,
    fs_vent,
    set_percentile=mb_percentile,
)

# Define the time axes
t_emg = [i/fs_emg for i in range(len(y_emg[0, :]))]
t_vent = [i/fs_vent for i in range(len(y_vent[0, :]))]

# Default settings for window of interest
# manoeuvres (Pocc)
# t_start_default = t_vent[-1]-61
# t_end_default = t_vent[-1]-1
t_start_default = t_vent[0]
t_end_default = t_vent[-1]

del data_emg_samples, data_vent_samples, data_emg, data_vent

btn_plt_raw = widgets.Dropdown(
    options=['Yes', 'No'],
    value='No',
    description='Plot raw data?',
    parasabled=False,
)
display(btn_plt_raw)

In [ ]:
# Plot the raw data if wanted
if btn_plt_raw.value == 'Yes':
 
    fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6), sharex=True)

    axis[0, 0].grid(True)
    axis[0, 0].plot(t_emg, y_emg[0])
    axis[0, 0].set(title='sEMG leads')
    axis[0, 0].set_ylabel('ECG (uV)')

    axis[1, 0].grid(True)
    axis[1, 0].plot(t_emg, y_emg[1])
    axis[1, 0].set_ylabel('sEMGdi (uV)')
    axis[1, 0].set_xlabel('t (s)')

    axis[2, 0].grid(True)
    axis[2, 0].plot(t_emg, y_emg[2])
    axis[2, 0].set_ylabel('sEMGpara (uV)')
    axis[2, 0].set_xlabel('t (s)')

    axis[0, 1].set(title='Ventilator data')
    axis[0, 1].grid(True)
    axis[0, 1].plot(t_vent, y_vent[0])
    axis[0, 1].set_ylabel('Paw (cmH2O)')

    axis[1, 1].grid(True)
    axis[1, 1].plot(t_vent, y_vent[1])
    axis[1, 1].set_ylabel('F (L/min)')

    axis[2, 1].grid(True)
    axis[2, 1].plot(t_vent, y_vent[2])
    axis[2, 1].set_ylabel('V (mL)')
    axis[2, 1].set_xlabel('t (s)')

## 3. Select the time window of interest

In [ ]:
start = t_start_default
end = t_end_default

start_s = int(float(start)* fs_emg)
end_s = min([int(float(end)*fs_emg), len(y_emg[0,:])-1])
start_vent_s = int(float(start)* fs_vent)
end_vent_s = min(
    [int(float(end)* fs_vent), len(y_vent[0,:])-1]
)

fig_w = (int(end_vent_s)-int(start_vent_s))//(fs_vent*80)*12

## 4. Pre-process the sEMGdi

In [ ]:
# Eliminate the baseline wander from the data using a band-pass filter
bd_filtered_file_data = filt.emg_bandpass_butter_sample(
    y_emg, 1, 500, 2048, output='sos')
    
processed_data_emg_di = bd_filtered_file_data[1]
processed_data_emg_para = bd_filtered_file_data[2]

del y_emg

In [ ]:
# Gating settings

# Window size to plot the gating results from
plot_window = 5         

# QRS peak detection settings
# peak_fraction = 0.30 (default value)
peak_fraction = gate_threshold_default     # Minimal relative peak height
peak_width = 0.001
peak_dist = int(fs_emg/3)

# Shift of gated in samples relative to detected peaks      
# ECG_shift = -10 (default value)
ECG_shift = gate_ECG_shift_default

# Gate width in seconds
# gate_width = 0.10 (default value)
gate_width = gate_width_default
# gate_width = 0.15

Apply QRS gating

In [ ]:
# First run of QRS gating

# Detect ECG peaks on 1Hz high-pass filtered signals
ecg = bd_filtered_file_data[0]
ecg_rms = evl.full_rolling_rms(ecg, 10)
max_ecg_rms = max(
    ecg_rms[int(start_s):int(start_s+plot_window*fs_emg)])
min_ecg_rms = min(
    ecg_rms[int(start_s):int(start_s+plot_window*fs_emg)])
peak_height = peak_fraction*(max_ecg_rms - min_ecg_rms)

ECG_peaks, properties  = scipy.signal.find_peaks(
    ecg_rms, 
    height=peak_height, 
    width=peak_width*fs_emg, 
    distance=peak_dist
)

# The RMS filter is not centred. Therefore, shift the ECG peak locations
ECG_peaks = ECG_peaks - ECG_shift

# Filter EXG signals 20Hz high pass
ECG_h20 = filt.emg_bandpass_butter_sample(ecg, 20, 500, 2048, output='sos')
emg_di_h20 = filt.emg_bandpass_butter_sample(
    processed_data_emg_di, 20, 500, 2048, output='sos')
emg_para_h20 = filt.emg_bandpass_butter_sample(
    processed_data_emg_para, 20, 500, 2048, output='sos')

# Gate ECG and EMG signal
# Fill methods 0: Zeros, 1: Interpolate start-end, 2: Average prior data, 
# 3: Moving average
gate_fill_method = 3    
gate_width_samples = int(gate_width*fs_emg)

gate_samples = list()
for i in range(len(ECG_peaks)):
    for k in range(
        int(ECG_peaks[i]-gate_width_samples/2),
        int(ECG_peaks[i]+gate_width_samples/2)
    ):
        gate_samples.append(k)

ECG_gated = ecg_rm.gating(
    ECG_h20, ECG_peaks, gate_width=gate_width_samples, method=3)
emg_di_gated = ecg_rm.gating(
    emg_di_h20, ECG_peaks, gate_width=gate_width_samples, method=3)
emg_para_gated = ecg_rm.gating(
    emg_para_h20, ECG_peaks, gate_width=gate_width_samples, method=3)


btn_gate_twice = widgets.Dropdown(
    options=['Yes', 'No'],
    value='Yes' if gate_twice else 'No',
    description='Gate twice?',
    parasabled=False,
)
display(btn_gate_twice)

In [ ]:
# Second run of QRS gating

if btn_gate_twice.value == 'Yes':
    gate_twice = True
else:
    gate_twice = False
    
# Second run of QRS gating, if selected with gate_twice == True
# This is usefull, e.g., in case of splitted QRS complex or pacing
if gate_twice == True:
    # Detect ECG peaks on 1Hz high-pass filtered signals
    ecg = ECG_gated
    ecg_rms = evl.full_rolling_rms(ecg, 10)
    max_ecg_rms = max(
        ecg_rms[int(start_s):int(start_s+plot_window*fs_emg)])
    min_ecg_rms = min(
        ecg_rms[int(start_s):int(start_s+plot_window*fs_emg)])
    peak_height = peak_fraction*(max_ecg_rms - min_ecg_rms)

    ECG_peaks, properties  = scipy.signal.find_peaks(
        ecg_rms, 
        height=peak_height, 
        width=peak_width*fs_emg, 
        distance=peak_dist
    )
    # The RMS filter is not centred. Therefore, shift the ECG peak locations
    ECG_peaks = ECG_peaks - ECG_shift

    # Gate ECG and EMG signal
    # Fill methods: 0: Zeros, 1: Interpolate start-end, 2: Average prior data, 
    # 3: Moving average
    gate_fill_method = 3    
    gate_width_samples = int(gate_width*fs_emg)

    gate_samples = list()
    for i in range(len(ECG_peaks)):
        for k in range(
            int(ECG_peaks[i]-gate_width_samples/2),
            int(ECG_peaks[i]+gate_width_samples/2)
        ):
            gate_samples.append(k)

    ECG_gated = ecg_rm.gating(
        ECG_gated, ECG_peaks, gate_width=gate_width_samples, method=3)
    emg_di_gated = ecg_rm.gating(
        emg_di_gated, ECG_peaks, gate_width=gate_width_samples, method=3)
    emg_para_gated = ecg_rm.gating(
        emg_para_gated, ECG_peaks, gate_width=gate_width_samples, method=3)

In [ ]:
btn_plt_gated = widgets.Dropdown(
    options=['Yes', 'No'],
    value='No',
    description='Plot gated data?',
    parasabled=False,
)
display(btn_plt_gated)

In [ ]:
# Plot gating result if wanted
if btn_plt_gated.value == 'Yes':
    fig, axis = plt.subplots(nrows=5, ncols=1, figsize=(12, 6), sharex=True)
    start_i = int(start_s)
    end_i = int(start_s+plot_window*2048)
    axis[0].grid(True)
    axis[0].plot(t_emg[start_i:end_i], bd_filtered_file_data[0, start_i:end_i])
    axis[0].set_ylabel('raw ECG (uV)')
    axis[0].set(title='leads in EMG')

    axis[1].grid(True)
    axis[1].plot(t_emg[start_i:end_i], ECG_gated[start_i:end_i])
    axis[1].set_ylabel('ECG gated (uV)')

    axis[2].grid(True)
    axis[2].plot(t_emg[start_i:end_i], emg_di_gated[start_i:end_i])
    axis[2].set_ylabel('EMGdi gated (uV)')

    axis[3].grid(True)
    axis[3].plot(t_emg[start_i:end_i], emg_para_gated[start_i:end_i])
    axis[3].set_ylabel('EMGpara gated (uV)')

    axis[4].grid(True)
    axis[4].plot(t_emg[start_i:end_i], ecg_rms[start_i:end_i])
    axis[4].set_ylabel('ECG rms (uV)')
    axis[4].hlines(
        y=peak_height, 
        xmin=t_emg[start_i], 
        xmax=t_emg[end_i],
        color = "C1"
    )
    axis[4].set_xlabel('t (s)')

    for idx in range(len(ECG_peaks)):
        if ((ECG_peaks[idx] > int(start_s)) 
            & (ECG_peaks[idx] < int(start_s+plot_window*2048))):
            axis[4].plot(t_emg[ECG_peaks[idx]], ecg_rms[ECG_peaks[idx]],'rx')


## 5. Calculate the sEMG envelope (sEAdi & sEApara)

In [ ]:
# Calculate the moving-RMS over the sEMG signal
# RMS_window_ms = 200 (default value)
RMS_window_ms = RMS_window_ms_default

RMS_windows_samp = int(RMS_window_ms / 1000 *  fs_emg)

# Pad the EMG signal before gating to get a centralised, rather than a causal
# moving RMS signal
pre_samp = int(np.ceil(RMS_windows_samp/2))
post_samp = int(np.floor(RMS_windows_samp/2))
padding_pre = np.zeros((pre_samp,))
padding_end = np.zeros((post_samp,))

emg_di_gated_padded = np.concatenate((padding_pre, emg_di_gated, padding_end))
sEAdi_padded = evl.full_rolling_rms(emg_di_gated_padded, RMS_windows_samp)
sEAdi = sEAdi_padded[:-RMS_windows_samp]

emg_para_gated_padded = np.concatenate(
    (padding_pre, emg_para_gated, padding_end))
sEApara_padded = evl.full_rolling_rms(emg_para_gated_padded, RMS_windows_samp)
sEApara = sEApara_padded[:-RMS_windows_samp]

btn_plt_RMS = widgets.Dropdown(
    options=['Yes', 'No'],
    value='No',
    description='Plot RMS data?',
    parasabled=False,
)
display(btn_plt_RMS)

In [ ]:
# Plot RMS results if wanted
if btn_plt_RMS.value == 'Yes':

    fig, axis = plt.subplots(nrows=5, ncols=1, figsize=(12, 6), sharex=True)
    axis[0].set(title='leads in EMG')
    axis[0].grid(True)
    axis[0].plot(t_emg[int(start_s):int(end_s)], 
                processed_data_emg_di[int(start_s):int(end_s)])
    axis[0].plot(t_emg[int(start_s):int(end_s)], sEAdi[int(start_s):int(end_s)])
    axis[0].set_ylabel('sEMGdi (uV)')

    axis[1].grid(True)
    axis[1].plot(t_emg[int(start_s):int(end_s)], sEAdi[int(start_s):int(end_s)])
    axis[1].set_ylabel('sEAdi (uV)')

    axis[2].grid(True)
    axis[2].plot(t_emg[int(start_s):int(end_s)], 
                processed_data_emg_para[int(start_s):int(end_s)])
    axis[2].plot(t_emg[int(start_s):int(end_s)], sEApara[int(start_s):int(end_s)])
    axis[2].set_ylabel('sEMGpara (uV)')

    axis[3].grid(True)
    axis[3].plot(t_emg[int(start_s):int(end_s)], sEApara[int(start_s):int(end_s)])
    axis[3].set_ylabel('sEApara (uV)')

    axis[4].grid(True)
    axis[4].plot(t_vent[int(start_vent_s):int(end_vent_s)], 
                y_vent[2][int(start_vent_s):int(end_vent_s)])
    axis[4].set_ylabel('Vvent (mL)')
    axis[4].set_xlabel('t (s)')

## 6. Calculate the Paw and sEAdi moving baselines

In [ ]:
# Baseline windows
baseline_W_emg = int(7.5 * fs_emg)  # window length
baseline_W_vent = int(7.5 * fs_vent)  # window length

augmented_perc = 25

In [ ]:
# 1. Calculate the moving baseline over the Paw signal over a 7.5s window
P_rolling_baseline = moving_baseline(
    emg_env=y_vent[0, int(start_vent_s):int(end_vent_s)], 
    window_s=baseline_W_vent,
    step_s=fs_vent//5,
    set_percentile=33)

In [ ]:
# Augmented moving baseline for EMGdi signals for baseline crossing detection

# 2.a. Calculate the "default" moving baseline over the sEAdi data over a 7.5s 
#     window
sEAdi_rolling_baseline = moving_baseline(
    emg_env=sEAdi[int(start_s):int(end_s)],
    window_s=baseline_W_emg,
    step_s=fs_emg//5,
    set_percentile=33)

# 2.b. Rolling standard deviation and mean over 7.5s window
# 2.c. Augmented signal: EMG + abs([dEMG/dt]_smoothed)
# 2.d. Run the moving median filter over the augmented signal to obtain the 
#       baseline
(sEAdi_aug_rolling_baseline, 
 di_baseline_mean, 
 di_baseline_std, _) = slopesum_baseline(
    emg_env = sEAdi[int(start_s):int(end_s)],
    window_s = int(fs_emg*7.5), 
    step_s = fs_emg//5,
    fs = fs_emg,
    set_percentile=33,
    augm_percentile=25)

In [ ]:
# Augmented moving baseline for EMGpara signals for baseline crossing detection

# 3.a. Calculate the "default" moving baseline over the sEApara data over a 7.5s 
#     window
sEApara_rolling_baseline = moving_baseline(
    emg_env=sEApara[int(start_s):int(end_s)], 
    window_s=baseline_W_emg,
    step_s=fs_emg//5,
    set_percentile=33)

# 3.b. Rolling standard deviation and mean over 7.5s window
# 3.c. Augmented signal: EMG + abs([dEMG/dt]_smoothed)
# 3.d. Run the moving median filter over the augmented signal to obtain the 
#       baseline

(sEApara_aug_rolling_baseline, 
 para_baseline_mean,
 para_baseline_std, _) = slopesum_baseline(
    emg_env=sEApara[int(start_s):int(end_s)], 
    window_s = int(fs_emg*7.5), 
    step_s = fs_emg//5,
    fs = fs_emg,
    set_percentile=33,
    augm_percentile=25)

## 7. Identify sEAdi, sEApara, and ventilator peaks of interest

In [ ]:
# EMG peak detection parameters:

# Threshold peak height as fraction of max peak height 
# sEAdi_prominence_factor = 0.5 (default value)
sEAdi_prominence_factor = 0.5
# sEAdi_prominence_factor = 1.0

# sEApara_prominence_factor = 0.5 (default value)
sEApara_prominence_factor = 0.5
# sEApara_prominence_factor = 1.0

emg_peak_width = 0.2
emg_peak_distance = 40/60

# Ventilator delay
# time_shift_default = 0.5 (default value)
time_shift = time_shift_default = 0.5 - RMS_window_ms/1000/2

# Breath & Tidal volume detection
# TV_percentile_default = 75 (default value)
TV_percentile = 75 
# TV_fraction_default = 0.2 (default value)
TV_fraction = 0.2 

print("The sEAdi detection threshold is set at:\n", 
      100*sEAdi_prominence_factor, '%')
print("The sEApara detection threshold is set at:\n", 
      100*sEApara_prominence_factor, '%')

sEAdi peak detection

In [ ]:
# Find diaphragm sEAdi peaks and baseline crossings using the new baseline

y_di_RMS = sEAdi[int(start_s):int(end_s)]
treshold = 0
width = int(emg_peak_width * fs_emg)
prominence = sEAdi_prominence_factor * \
    (np.nanpercentile(y_di_RMS - sEAdi_rolling_baseline, 75) 
     + np.nanpercentile(y_di_RMS - sEAdi_rolling_baseline, 50))
distance = int(emg_peak_distance * fs_emg)
sEAdi_peaks, properties = scipy.signal.find_peaks(
    y_di_RMS-sEAdi_rolling_baseline, 
    height=treshold, 
    prominence=prominence, 
    width=width, 
    distance=distance
)

# Find peaks based on augmented baseline

# Amplify the augmented moving baseline according to the Coefficient of 
# Variation (CoV = std/mean)
if len(sEAdi_peaks) > 0:
    sEAdi_rolling_baseline_aug = sEAdi_rolling_baseline * (1+ 
        np.nanmedian(y_di_RMS[sEAdi_peaks]) * di_baseline_std 
        / di_baseline_mean **2
    )
else:
    sEAdi_rolling_baseline_aug = sEAdi_rolling_baseline * (1+ 
        np.nanmedian(y_di_RMS) * di_baseline_std 
        / di_baseline_mean **2
    )

(peak_idxs, peak_start_idxs, 
 peak_end_idxs, valid_starts_bools,
 valid_ends_bools, valid_peaks) = onoffpeak_baseline_crossing(
    y_di_RMS,
    sEAdi_rolling_baseline_aug,
    sEAdi_peaks
)

sEAdi_starts = peak_start_idxs[valid_peaks]
sEAdi_peaks = sEAdi_peaks[valid_peaks]
sEAdi_ends = peak_end_idxs[valid_peaks]

#  ------------------------------

# Detect the sEAdi on- and offsets based on extrapolated slopes
# ma_window = fs_emg//4
# signal = pd.Series(y_di_RMS - sEAdi_rolling_baseline)
# signal_MA = signal.rolling(window=ma_window, center=True).mean().values

# (peak_start_idxs, peak_end_idxs,
#  valid_starts_bools, valid_ends_bools, 
#  valid_peaks) = onoffpeak_slope_extrapolation(
#     signal_MA,
#     fs_emg,
#     sEAdi_peaks,
#     slope_window_s=fs_emg//10
# )
# sEAdi_starts = peak_start_idxs[valid_peaks]
# sEAdi_peaks = sEAdi_peaks[valid_peaks]
# sEAdi_ends = peak_end_idxs[valid_peaks]



In [ ]:
# Find parasternal sEApara peaks and baseline crossings using the new baseline

y_para_RMS = sEApara[int(start_s):int(end_s)]
treshold = 0
width = int(emg_peak_width * fs_emg)
prominence = sEApara_prominence_factor * \
    (np.nanpercentile(y_para_RMS - sEApara_rolling_baseline, 75) 
     + np.nanpercentile(y_para_RMS - sEApara_rolling_baseline, 50))
distance = int(emg_peak_distance * fs_emg)
sEApara_peaks, properties = scipy.signal.find_peaks(
    y_para_RMS-sEApara_aug_rolling_baseline, 
    height=treshold,
    prominence=prominence, 
    width=width, 
    distance=distance
)

# Find peaks based on augmented baseline

# Amplify the augmented moving baseline according to the Coefficient of 
# Variation (CoV = std/mean)
if len(sEApara_peaks) > 0:
    sEApara_rolling_baseline_aug = sEApara_rolling_baseline * (1+ 
        np.nanmedian(y_para_RMS[sEApara_peaks]) * para_baseline_std 
        / para_baseline_mean **2
    )
else:
    sEApara_rolling_baseline_aug = sEApara_rolling_baseline * (1+ 
        np.nanmedian(y_para_RMS) * para_baseline_std 
        / para_baseline_mean **2
    )

(peak_idxs, peak_start_idxs, 
 peak_end_idxs, valid_starts_bools,
 valid_ends_bools, valid_peaks) = onoffpeak_baseline_crossing(
    y_para_RMS,
    sEApara_rolling_baseline_aug,
    sEApara_peaks
)

sEApara_starts = peak_start_idxs[valid_peaks]
sEApara_peaks = sEApara_peaks[valid_peaks]
sEApara_ends = peak_end_idxs[valid_peaks]

# --------------------------------

# # Detect the sEAdi on- and offsets based on slope extrapolation
# ma_window = fs_emg//4
# signal = pd.Series(y_para_RMS - sEApara_rolling_baseline)
# signal_MA = signal.rolling(window=ma_window, center=True).mean().values

# (peak_start_idxs, peak_end_idxs,
#  valid_starts_bools, valid_ends_bools, 
#  valid_peaks) = onoffpeak_slope_extrapolation(
#     signal_MA,
#     fs_emg,
#     sEApara_peaks,
#     slope_window_s=fs_emg//2
# )
# sEApara_starts = peak_start_idxs[valid_peaks]
# sEApara_peaks = sEApara_peaks[valid_peaks]
# sEApara_ends = peak_end_idxs[valid_peaks]


Pneumatic data detection

In [ ]:
# Detect pneumatic peaks and the set PEEP level
# Find end-expiration samples by finding the minimal values in V_vent
V_ei_PKS_all, _ = scipy.signal.find_peaks(y_vent[2])

# Detect tidal peaks with a volume at least 20% of the upper quartile TV
TV_75_pct = np.percentile(y_vent[2, V_ei_PKS_all], TV_percentile)
threshold = TV_fraction * TV_75_pct
V_ei_PKS, _ = scipy.signal.find_peaks(y_vent[2], height=threshold)

if len(sEApara_starts) > 0:
    while sEApara_starts[0]/fs_emg*fs_vent > V_ei_PKS[0]:
        # Eliminate tidal peaks before the first full sEApara peak
        V_ei_PKS = np.delete(V_ei_PKS, 0)

# Find the set PEEP level
# Find end-expiration samples by finding the minimal values in V_vent
V_ee_PKS, _ = scipy.signal.find_peaks(-y_vent[2])

# Eliminate end tidal volumes before the first tidal volume peak
while V_ee_PKS[0] < V_ei_PKS[0]:
    V_ee_PKS = np.delete(V_ee_PKS, 0)

# Eliminate double end tidal volumes before the first tidal volume peak
for idx in range(len(V_ei_PKS)-1):
    Vee_idxs_tmp = V_ee_PKS[(V_ee_PKS > V_ei_PKS[idx]) 
                        & (V_ee_PKS < V_ei_PKS[idx+1])]
    stay_idx = np.argmin(y_vent[2, Vee_idxs_tmp])
    if len(Vee_idxs_tmp) > 1:
        stay_idx
        Vee_idxs_tmp = np.delete(Vee_idxs_tmp, stay_idx)

        for _, peak_idx in enumerate(Vee_idxs_tmp):
            V_ee_PKS = np.delete(V_ee_PKS, np.argwhere(peak_idx == V_ee_PKS))

# Eliminate tidal peak from which the end-expiration is not detected
while V_ee_PKS[-1] < V_ei_PKS[-1]:
    V_ei_PKS = np.delete(V_ei_PKS, -1)


# The remaining tidal volumes:
print("The detected tidal volumes are:\n", 
      'Median: ' + str(np.nanmedian(y_vent[2, V_ei_PKS])), ' mL\n'
      'Min: ' + str(np.nanmin(y_vent[2, V_ei_PKS])), ' mL\n'
      'Max: ' + str(np.nanmax(y_vent[2, V_ei_PKS])), ' mL\n')

# Calculate PEEP as the median value of Paw at end-expiration
PEEP_set = np.round(np.median(y_vent[0, V_ee_PKS]))

print("The detected PEEP level is:\n", PEEP_set, ' cmH2O \n')

# Calculate PS as the median value of the driving pressure just before end-
# inspiration, rounded to a multiple of 3 cmH2O
Paw_PKS, _ = scipy.signal.find_peaks(y_vent[0] - PEEP_set, prominence=2.5)
i_delta = fs_vent // 5
P_peaks_median = [np.nanmedian(
    y_vent[0, Paw_PKS[idx]-i_delta:Paw_PKS[idx]+i_delta]
           ) for idx in range(len(Paw_PKS))]
# PS_set = np.round(np.median(P_peaks_median - PEEP_set) / 3) * 3
PS_set = np.round(np.nanmedian(P_peaks_median - PEEP_set))

print("The detected PS level is:\n", PS_set, ' cmH2O')

# Link pneumatic and EMG peaks

In [ ]:
# Convert ventilator samples to EMG sample rate
P_peaks_converted = Paw_PKS / fs_vent * fs_emg
V_peaks_converted = V_ei_PKS / fs_vent * fs_emg

# Link sEAdi peak closest to pneumatic peaks
sEAdi_links = np.zeros((len(sEAdi_peaks), 2), dtype=int) - 1

for idx in range(len(sEAdi_peaks)):
    # Linked pressure peak
    sEAdi_links[idx, 0] = np.argmin(
        np.abs(sEAdi_peaks[idx] + time_shift_default * fs_emg 
               - P_peaks_converted)
    )

    # Linked volume peak
    sEAdi_links[idx, 1] = np.argmin(
        np.abs(sEAdi_peaks[idx] + time_shift_default * fs_emg 
               - V_peaks_converted)
    )

# Link sEApara peak closest to pneumatic peaks
sEApara_links = np.zeros((len(sEApara_peaks), 2), dtype=int) - 1

for idx in range(len(sEApara_peaks)):
    # Linked pressure peak
    sEApara_links[idx, 0] = np.argmin(
        np.abs(sEApara_peaks[idx] + time_shift_default * fs_emg 
               - P_peaks_converted)
    )

    # Linked volume peak
    sEApara_links[idx, 1] = np.argmin(
        np.abs(sEApara_peaks[idx] + time_shift_default * fs_emg 
               - V_peaks_converted)
    )

# Convert EMGsamples to ventilator sample rate
sEAdi_peaks_converted = sEAdi_peaks / fs_emg * fs_vent
sEApara_peaks_converted = sEApara_peaks / fs_emg * fs_vent

# Link pressure peaks to the closest EMG peaks
Paw_links = np.zeros((len(Paw_PKS), 2), dtype=int) - 1

for idx in range(len(Paw_PKS)):
    # Linked pressure peak
    if len(sEAdi_peaks) > 0:
        Paw_links[idx, 0] = np.argmin(
            np.abs(sEAdi_peaks_converted + time_shift_default * fs_vent
                   - Paw_PKS[idx])
        )

    # Linked volume peak
    if len(sEApara_peaks) > 0:
        Paw_links[idx, 1] = np.argmin(
            np.abs(sEApara_peaks_converted 
                   + time_shift_default * fs_vent
                   - Paw_PKS[idx])
        )

# Link pressure peaks to the closest EMG peaks
Vvent_links = np.zeros((len(V_ei_PKS), 2), dtype=int) - 1

for idx in range(len(V_ei_PKS)):
    # Linked pressure peak
    if len(sEAdi_peaks) > 0:
        Vvent_links[idx, 0] = np.argmin(
            np.abs(sEAdi_peaks_converted 
                   + time_shift_default * fs_vent
                   - V_ei_PKS[idx])
        )

    # Linked volume peak
    if len(sEApara_peaks) > 0:
        Vvent_links[idx, 1] = np.argmin(
            np.abs(sEApara_peaks_converted 
                   + time_shift_default * fs_vent
                   - V_ei_PKS[idx])
        )

In [ ]:
len(sEAdi_starts), len(sEAdi_peaks), len(sEAdi_ends)

In [ ]:
# Data sanity check: Make sure sEAdi peaks have been detected correctly
# if (any(sEAdi_starts == 0) or any(sEAdi_peaks == 0)  
#     or any(sEAdi_ends == 0)):
#     print('The number of detected sEAdi onsets, peaks, offsets using the new '
#           + 'moving baseline method, is not the same, please examine the data. '
#           + 'Selected another window or file, accordingly')
min_len = np.min([len(sEAdi_starts), len(sEAdi_peaks), len(sEAdi_ends)])
if (len(sEAdi_starts)-min_len != 0 or len(sEAdi_peaks)-min_len != 0
    or len(sEAdi_ends)-min_len != 0):
    print('The number of detected sEAdi onsets, peaks, offsets using the new '
          + 'moving baseline method, is not the same, please examine the data. '
          + 'Selected another window or file, accordingly')
else:
    print(str(len(sEAdi_peaks)) + ' sEAdi peak(s) were detected using the '
          'new moving baseline method. You may continue!')

In [ ]:
# Data sanity check: Make sure sEApara peaks have been detected correctly
# if (any(sEApara_starts == 0) or any(sEApara_peaks == 0)  
#     or any(sEApara_ends == 0)):
#     print('The number of detected sEApara onsets, peaks, offsets using the new '
#           + 'moving baseline method, is not the same, please examine the data. '
#           + 'Selected another window or file, accordingly')
min_len = np.min([len(sEApara_starts), len(sEApara_peaks), len(sEApara_ends)])
if (len(sEApara_starts)-min_len != 0 or len(sEApara_peaks)-min_len != 0
    or len(sEApara_ends)-min_len != 0):
    print('The number of detected sEApara onsets, peaks, offsets using the new '
        + 'moving baseline method, is not the same, please examine the data. '
        + 'Selected another window or file, accordingly')
else:
    print(str(len(sEApara_peaks)) + ' sEApara peak(s) were detected using the '
          'new moving baseline method. You may continue!')

## 8. Plot the detected Paw, Vvent, sEAdi and sEApara peaks

In [ ]:
# Plot entire selected time window with indicated peaks
# Paw:
fig_1, axis = plt.subplots(nrows=4, ncols=1, figsize=(12, 6), sharex=True)

Paw_sel = y_vent[0, start_vent_s:end_vent_s+1]
axis[0].grid(True)
#   Paw signal
axis[0].plot([y / fs_vent for y in range(len(Paw_sel))], Paw_sel)
#   Paw moving baseline
axis[0].plot([y / fs_vent for y in range(len(P_rolling_baseline))],
             P_rolling_baseline)
#   Detected peaks, and on- and offsets 
axis[0].plot(Paw_PKS/fs_vent, Paw_sel[Paw_PKS], "x", color="r")
axis[0].set_ylabel('Paw (cmH2O)')

# V_vent:
axis[1].grid(True)
V_vent = y_vent[2, int(start_vent_s):int(end_vent_s)+1]
axis[1].plot([y / fs_vent for y in range(len(V_vent))], V_vent)
axis[1].plot(V_ei_PKS/fs_vent, V_vent[V_ei_PKS], "x", color="r")
axis[1].plot(V_ee_PKS/fs_vent, V_vent[V_ee_PKS], "*", color="g")
axis[1].set_ylabel('V (mL)')

# sEAdi:
axis[2].grid(True)
N_samp = len(sEAdi[int(start_s):int(end_s)])
# Normalised ECG for reference
if len(sEAdi_peaks) > 0:
    axis[2].plot(np.array(t_emg[start_s:end_s]), 
                        ecg_rms[start_s:end_s]/
                        max(ecg_rms[start_s:end_s])*
                        np.percentile(y_di_RMS[sEAdi_peaks], 75), 
                        'tab:gray', linewidth=0.5)
else:
    axis[2].plot(np.array(t_emg[start_s:end_s]), 
                    ecg_rms[start_s:end_s]/
                    max(ecg_rms[start_s:end_s])*
                    np.percentile(y_di_RMS, 75), 
                    'tab:gray', linewidth=0.5)
#   sEAdi signal
axis[2].plot([y / fs_emg for y in range(N_samp)],
             sEAdi[int(start_s):int(end_s)])
#   Rolling baseline
# axis[2].plot([y / fs_emg 
#               for y in range(len(sEAdi_rolling_baseline))],
#              sEAdi_rolling_baseline[:int(end_s)-int(start_s)])
axis[2].plot([y / fs_emg 
              for y in range(len(sEAdi_rolling_baseline_aug))],
             sEAdi_rolling_baseline_aug[:int(end_s)-int(start_s)])
#   Detected peaks, and on- and offsets 
axis[2].plot(sEAdi_peaks/fs_emg,
             y_di_RMS[sEAdi_peaks], "x", color="r")
axis[2].plot(sEAdi_starts/ fs_emg,
             y_di_RMS[sEAdi_starts], '*r')
axis[2].plot(sEAdi_ends / fs_emg,
             y_di_RMS[sEAdi_ends], '*r')

axis[2].set_ylabel('sEAdi (uV)')
axis[2].set_xlabel('t (s)')
if len(sEAdi_peaks) > 0:
    axis[2].set_ylim([0, 1.4*np.percentile(y_di_RMS[sEAdi_peaks], 95)])

# sEApara:
axis[3].grid(True)
N_samp = len(sEApara[int(start_s):int(end_s)])
# Normalised ECG for reference
if len(sEApara_starts) > 0:
    axis[3].plot(np.array(t_emg[start_s:end_s]), 
                        ecg_rms[start_s:end_s]/
                        max(ecg_rms[start_s:end_s])*
                        np.percentile(y_para_RMS[sEApara_peaks], 75), 
                        'tab:gray', linewidth=0.5)
else:
    axis[3].plot(np.array(t_emg[start_s:end_s]), 
                        ecg_rms[start_s:end_s]/
                        max(ecg_rms[start_s:end_s])*
                        np.percentile(y_para_RMS, 75), 
                        'tab:gray', linewidth=0.5)
#   sEApara signal
axis[3].plot([y / fs_emg for y in range(N_samp)],
             sEApara[int(start_s):int(end_s)])
#   Rolling baseline
# axis[3].plot([y / fs_emg 
#               for y in range(len(sEApara_rolling_baseline))],
#              sEApara_rolling_baseline[:int(end_s)-int(start_s)])
axis[3].plot([y / fs_emg 
              for y in range(len(sEApara_rolling_baseline_aug))],
             sEApara_rolling_baseline_aug[:int(end_s)-int(start_s)])
#   Detected peaks, and on- and offsets 
axis[3].plot(sEApara_peaks/fs_emg,
             y_para_RMS[sEApara_peaks], "x", color="r")
axis[3].plot(sEApara_starts/ fs_emg,
             y_para_RMS[sEApara_starts], '*r')
axis[3].plot(sEApara_ends / fs_emg,
             y_para_RMS[sEApara_ends], '*r')

axis[3].set_ylabel('sEApara (uV)')
axis[3].set_xlabel('t (s)')
if len(sEApara_starts) > 0:
    axis[3].set_ylim([0, 1.4*np.percentile(y_para_RMS[sEApara_peaks], 95)])

## 9. Calculate the ETPdi and ETPpara per peak

In [ ]:
# Calculate the electrical time product over sEAdi (ETPdi)

ETP_di_occs = np.zeros((len(sEAdi_peaks),))
y_di_min = np.zeros((len(sEAdi_peaks),))
ETP_di_baseline = np.zeros((len(sEAdi_peaks),))
ETP_di_baseline_old = np.zeros((len(sEAdi_peaks),))
noise_heights_di = np.zeros((len(sEAdi_peaks),))
for idx in range(len(sEAdi_peaks)):
    start_i = sEAdi_starts[idx]
    end_i = sEAdi_ends[idx]

    # Area under the baseline (AUB)
    baseline_start_i = max([0, sEAdi_peaks[idx] - 5*fs_emg])
    baseline_end_i = min(
        [len(y_di_RMS) - 1, sEAdi_peaks[idx] + 5*fs_emg])
    y_di_min[idx] = min(y_di_RMS[baseline_start_i:baseline_end_i])

    
    ETP_di_baseline[idx] = np.trapz(
        sEAdi_rolling_baseline_aug[start_i:end_i+1] 
        - y_di_min[idx], dx=1/fs_emg
    )
    # ETP_di_baseline[idx] = np.trapz(
    #     sEAdi_rolling_baseline[start_i:end_i+1] 
    #     - y_di_min[idx], dx=1/fs_emg
    # )

    # EMG Time Product (ETP) diaphragm
    # ETP_di_occs[idx] = np.trapz(
    #     y_di_RMS[start_i:end_i+1] 
    #     - sEAdi_rolling_baseline[start_i:end_i+1],
    #     dx=1/fs_emg
    # ) + ETP_di_baseline[idx]
    ETP_di_occs[idx] = np.trapz(
        y_di_RMS[start_i:end_i+1] 
        - sEAdi_rolling_baseline_aug[start_i:end_i+1],
        dx=1/fs_emg
    ) + ETP_di_baseline[idx]

    noise_heights_di[idx] = np.median(sEAdi_rolling_baseline[start_i:end_i])

In [ ]:
# Calculate the electrical time product over sEApara (ETPpara)

ETP_para_occs = np.zeros((len(sEApara_peaks),))
y_para_min = np.zeros((len(sEApara_peaks),))
ETP_para_baseline = np.zeros((len(sEApara_peaks),))
ETP_para_baseline_old = np.zeros((len(sEApara_peaks),))
noise_heights_para = np.zeros((len(sEApara_peaks),))
for idx in range(len(sEApara_peaks)):
    start_i = sEApara_starts[idx]
    end_i = sEApara_ends[idx]

    # Area under the baseline (AUB)
    baseline_start_i = max([0, sEApara_peaks[idx] - 5*fs_emg])
    baseline_end_i = min(
        [len(y_para_RMS) - 1, sEApara_peaks[idx] + 5*fs_emg])
    y_para_min[idx] = min(y_para_RMS[baseline_start_i:baseline_end_i])

    
    ETP_para_baseline[idx] = np.trapz(
        sEApara_rolling_baseline_aug[start_i:end_i+1] 
        - y_para_min[idx], dx=1/fs_emg
    )
    # ETP_para_baseline[idx] = np.trapz(
    #     sEApara_rolling_baseline[start_i:end_i+1] 
    #     - y_para_min[idx], dx=1/fs_emg
    # )

    # # EMG Time Product (ETP) parasternal
    # ETP_para_occs[idx] = np.trapz(
    #     y_para_RMS[start_i:end_i+1] 
    #     - sEApara_rolling_baseline[start_i:end_i+1],
    #     dx=1/fs_emg
    # ) + ETP_para_baseline[idx]
    ETP_para_occs[idx] = np.trapz(
        y_para_RMS[start_i:end_i+1] 
        - sEApara_rolling_baseline_aug[start_i:end_i+1],
        dx=1/fs_emg
    ) + ETP_para_baseline[idx]

    noise_heights_para[idx] = np.median(sEApara_rolling_baseline[start_i:end_i])

## 10. Evaluate the quality of the Pocc and sEAdi peaks

10.a Successively occluded breaths: N/A

10.b Pocc release quality: N/A

10.c Interpeak time for ECG, sEAdi and sEApara peaks 

In [ ]:
# Determine electrophysiological interpeak distance
t_delta_ecg_med = np.median(np.array(t_emg)[ECG_peaks[1:]] 
                            - np.array(t_emg)[ECG_peaks[:-1]])
t_delta_di_med = np.median(np.array(t_emg)[sEAdi_peaks[1:]] 
                           - np.array(t_emg)[sEAdi_peaks[:-1]])
t_delta_para_med = np.median(np.array(t_emg)[sEApara_peaks[1:]] 
                           - np.array(t_emg)[sEApara_peaks[:-1]])

10.d sEAdi and sEApara signal-to-noise ratio (SNR)

In [ ]:
# Calculate signal-to-noise ratios. 
#   As the true signal and noise are not known,
#   the SNR is estimated as the maximum divided by the median of the moving 
#   baseline in a 1 s window around the peak.

# SNR_di = feat.snr_pseudo(
#     src_signal=sEAdi[int(start_s):int(end_s)], 
#     peaks=sEAdi_peaks, 
#     baseline=sEAdi_rolling_baseline[:int(end_s)-int(start_s)]
# )

# SNR_para = feat.snr_pseudo(
#     src_signal=sEApara[int(start_s):int(end_s)], 
#     peaks=sEApara_peaks, 
#     baseline=sEApara_rolling_baseline[:int(end_s)-int(start_s)]
# )
SNR_di = feat.snr_pseudo(
    src_signal=sEAdi[int(start_s):int(end_s)], 
    peaks=sEAdi_peaks, 
    baseline=sEAdi_rolling_baseline_aug[:int(end_s)-int(start_s)]
)

SNR_para = feat.snr_pseudo(
    src_signal=sEApara[int(start_s):int(end_s)], 
    peaks=sEApara_peaks, 
    baseline=sEApara_rolling_baseline_aug[:int(end_s)-int(start_s)]
)

10.e Area under the baseline

In [ ]:
# Plot the Area Under the Baseline (AUB) 
# NB1 AUB is calculated above in step 9.
# NB2 AUB is plotted in cyan 

# Paw:
fig_2, axis = plt.subplots(nrows=4, ncols=1, figsize=(12, 6), sharex=True)

Paw_sel = y_vent[0, start_vent_s:end_vent_s+1]
axis[0].grid(True)
#   Paw signal
axis[0].plot([y / fs_vent for y in range(len(Paw_sel))], Paw_sel)
#   Paw moving baseline
axis[0].plot([y / fs_vent for y in range(len(P_rolling_baseline))],
             P_rolling_baseline)
#   Detected peaks, and on- and offsets 
axis[0].plot(Paw_PKS/fs_vent, Paw_sel[Paw_PKS], "x", color="r")
axis[0].set_ylabel('Paw (cmH2O)')

# V_vent:
axis[1].grid(True)
V_vent = y_vent[2, start_vent_s:end_vent_s+1]
axis[1].plot([y / fs_vent for y in range(len(V_vent))], V_vent)
axis[1].plot(V_ei_PKS/fs_vent, V_vent[V_ei_PKS], "x", color="r")
axis[1].plot(V_ee_PKS/fs_vent, V_vent[V_ee_PKS], "*", color="g")
axis[1].set_ylabel('V (mL)')

# sEAdi:
axis[2].grid(True)
N_samp = len(sEAdi[int(start_s):int(end_s)])
#   sEAdi signal
axis[2].plot([y / fs_emg for y in range(N_samp)],
             sEAdi[int(start_s):int(end_s)])
#   Rolling baseline
axis[2].plot([y / fs_emg for y in range(N_samp)],
             sEAdi_rolling_baseline_aug[:int(end_s)-int(start_s)])
# axis[2].plot([y / fs_emg for y in range(N_samp)],
#              sEAdi_rolling_baseline[:int(end_s)-int(start_s)])

# Plot sEAdi area under the baseline (AUB)
for idx in range(len(sEAdi_peaks)):
    start_i = sEAdi_starts[idx]
    end_i = sEAdi_ends[idx]
    
    #   Area under the baseline (AUB)
    axis[2].plot([t_emg[start_i], t_emg[end_i+1]], 
                      [y_di_min[idx], y_di_min[idx]], 'c')
    axis[2].plot([t_emg[start_i], t_emg[start_i]], 
                      [y_di_RMS[start_i], y_di_min[idx]], 'c')
    axis[2].plot([t_emg[end_i], t_emg[end_i]], 
                      [y_di_RMS[end_i], y_di_min[idx]], 'c')
    #   Detected peaks, and on- and offsets
    axis[2].plot(t_emg[sEAdi_peaks[idx]],
                      y_di_RMS[sEAdi_peaks[idx]], '*r')
    axis[2].plot(t_emg[start_i], y_di_RMS[start_i], '*r')
    axis[2].plot(t_emg[end_i], y_di_RMS[end_i], '*r')


axis[2].set_ylabel('sEAdi (uV)')
if len(sEAdi_peaks) > 0:
    axis[2].set_ylim([0, 1.2*np.percentile(y_di_RMS[sEAdi_peaks], 75)])

# sEApara:
axis[3].grid(True)
N_samp = len(sEApara[int(start_s):int(end_s)])
#   sEApara signal
axis[3].plot([y / fs_emg for y in range(N_samp)],
             sEApara[int(start_s):int(end_s)])
#   Rolling baseline
axis[3].plot([y / fs_emg for y in range(N_samp)],
             sEApara_rolling_baseline_aug[:int(end_s)-int(start_s)])
# axis[3].plot([y / fs_emg for y in range(N_samp)],
#              sEApara_rolling_baseline[:int(end_s)-int(start_s)])

# Plot sEApara area under the baseline (AUB)
for idx in range(len(sEApara_peaks)):
    start_i = sEApara_starts[idx]
    end_i = sEApara_ends[idx]
    
    #   Area under the baseline (AUB)
    axis[3].plot([t_emg[start_i], t_emg[end_i+1]], 
                      [y_para_min[idx], y_para_min[idx]], 'c')
    axis[3].plot([t_emg[start_i], t_emg[start_i]], 
                      [y_para_RMS[start_i], y_para_min[idx]], 'c')
    axis[3].plot([t_emg[end_i], t_emg[end_i]], 
                      [y_para_RMS[end_i], y_para_min[idx]], 'c')
    #   Detected peaks, and on- and offsets
    axis[3].plot(t_emg[sEApara_peaks[idx]],
                      y_para_RMS[sEApara_peaks[idx]], '*r')
    axis[3].plot(t_emg[start_i], y_para_RMS[start_i], '*r')
    axis[3].plot(t_emg[end_i], y_para_RMS[end_i], '*r')


axis[3].set_ylabel('sEApara (uV)')
axis[3].set_xlabel('t (s)')
if len(sEApara_peaks) > 0:
    axis[3].set_ylim([0, 1.2*np.percentile(y_para_RMS[sEApara_peaks], 75)])

10.e Waveform morphology

In [ ]:
# Evaluate sEAdi and sEApara peak quality according to waveform morphology
# NB1 The fitted bell curve is plotted in green 

# Define the bell-curve function
def func(x, a, b, c):
    return a * np.exp(-(x-b)**2 / c**2)

# Evaluate bell-curve error of sEApara peaks
ETP_di_bell_error = np.zeros((len(sEAdi_peaks),))
for idx in range(len(sEAdi_peaks)):
    start_i = sEAdi_starts[idx]
    end_i = sEAdi_ends[idx]
    
    baseline_start_i = max([0, sEAdi_peaks[idx] - 5*fs_emg])
    baseline_end_i = min(
        [len(y_di_RMS) - 1, sEAdi_peaks[idx] + 5*fs_emg])
    y_di_min[idx] = min(y_di_RMS[baseline_start_i:baseline_end_i])

    # Too little samples to fit parameters to --> Add additional datapoints
    if end_i - start_i < 3:
        plus_idx = 3 - (end_i - start_i)
    else:
        plus_idx = 0

    try:
        popt, pcov = curve_fit(func, 
                        t_emg[start_i:end_i+1], 
                        y_di_RMS[start_i:end_i+1]-y_di_min[idx],
                        bounds=([-np.inf, t_emg[sEAdi_peaks[idx]]-0.5, -np.inf], 
                                [np.inf, t_emg[sEAdi_peaks[idx]]+0.5, np.inf])
                        )
        

        ETP_di_bell_error[idx] = np.trapz(
            np.sqrt((y_di_RMS[start_i:end_i+1] - 
            (func(t_emg[start_i:end_i+1], *popt)+y_di_min[idx])) **2),
            dx=1/fs_emg
        )
    except(RuntimeError):
        ETP_di_bell_error[idx] = np.nan

# Evaluate bell-curve error of sEApara peaks
ETP_para_bell_error = np.zeros((len(sEApara_peaks),))
for idx in range(len(sEApara_peaks)):
    start_i = sEApara_starts[idx]
    end_i = sEApara_ends[idx]
    
    baseline_start_i = max([0, sEApara_peaks[idx] - 5*fs_emg])
    baseline_end_i = min(
        [len(y_para_RMS) - 1, sEApara_peaks[idx] + 5*fs_emg])
    y_para_min[idx] = min(y_para_RMS[baseline_start_i:baseline_end_i])

    # Too little samples to fit parameters to --> Add additional datapoints
    if end_i - start_i < 3:
        plus_idx = 3 - (end_i - start_i)
    else:
        plus_idx = 0
    
    try:
        popt, pcov = curve_fit(func, 
                        t_emg[start_i:end_i+1], 
                        y_para_RMS[start_i:end_i+1]-y_para_min[idx],
                        bounds=([-np.inf, t_emg[sEApara_peaks[idx]]-0.5, -np.inf], 
                                [np.inf, t_emg[sEApara_peaks[idx]]+0.5, np.inf])
                        )

        ETP_para_bell_error[idx] = np.trapz(
            np.sqrt((y_para_RMS[start_i:end_i+1] - 
            (func(t_emg[start_i:end_i+1], *popt)+y_para_min[idx])) **2),
            dx=1/fs_emg
        )
    except(RuntimeError):
        ETP_para_bell_error[idx] = np.nan

In [ ]:
print('Unable to fit bell curve to sEAdi in ' 
      + str(np.count_nonzero(np.isnan(ETP_di_bell_error))) + ' case(s).\n'
      'Unable to fit bell curve to sEApara in ' 
      + str(np.count_nonzero(np.isnan(ETP_para_bell_error))) + ' case(s).'
)

In [ ]:
# Baseline_variability

sEAdi_bl_mean = np.nanmean(sEAdi_rolling_baseline)
sEAdi_bl_std = np.nanstd(sEAdi_rolling_baseline)

sEApara_bl_mean = np.nanmean(sEApara_rolling_baseline)
sEApara_bl_std = np.nanstd(sEApara_rolling_baseline)

sEAdi_bl_std/sEAdi_bl_mean, sEApara_bl_std/sEApara_bl_mean

In [ ]:
# Plot sEAdi peak quality according to waveform morphology

# Define the bell-curve function
def func(x, a, b, c):
    return a * np.exp(-(x-b)**2 / c**2)

fig_3, axis = plt.subplots(nrows=4, ncols=1, figsize=(12, 6), sharex=True)

Paw_sel = y_vent[0, start_vent_s:end_vent_s+1]
axis[0].grid(True)
#   Paw signal
axis[0].plot([y / fs_vent for y in range(len(Paw_sel))], Paw_sel)
#   Paw moving baseline
axis[0].plot([y / fs_vent for y in range(len(P_rolling_baseline))],
             P_rolling_baseline)
#   Detected peaks, and on- and offsets 
axis[0].plot(Paw_PKS/fs_vent, Paw_sel[Paw_PKS], "x", color="r")
axis[0].set_ylabel('Paw (cmH2O)')

# V_vent:
axis[1].grid(True)
V_vent = y_vent[2, start_vent_s:end_vent_s+1]
axis[1].plot([y / fs_vent for y in range(len(V_vent))], V_vent)
axis[1].plot(V_ei_PKS/fs_vent, V_vent[V_ei_PKS], "x", color="r")
axis[1].plot(V_ee_PKS/fs_vent, V_vent[V_ee_PKS], "*", color="g")
axis[1].set_ylabel('V (mL)')

# sEAdi:
axis[2].grid(True)
N_samp = len(sEAdi[int(start_s):int(end_s)])
#   sEAdi signal
axis[2].plot([y / fs_emg for y in range(N_samp)],
             sEAdi[int(start_s):int(end_s)])
#   Rolling baseline
axis[2].plot([y / fs_emg for y in range(N_samp)],
             sEAdi_rolling_baseline[:int(end_s)-int(start_s)])
# axis[2].plot([y / fs_emg for y in range(N_samp)],
#              sEAdi_rolling_baseline_aug[:int(end_s)-int(start_s)])

# Plot bell-curve error of sEAdi peaks
for idx in range(len(sEAdi_peaks)):
    start_i = sEAdi_starts[idx]
    end_i = sEAdi_ends[idx]

    start_plot_i = max([0, start_i-fs_emg//4])
    end_plot_i = min([N_samp, end_i+fs_emg//4])
    
    baseline_start_i = max([0, sEAdi_peaks[idx] - 5*fs_emg])
    baseline_end_i = min(
        [len(y_di_RMS) - 1, sEAdi_peaks[idx] + 5*fs_emg])
    y_di_min[idx] = min(y_di_RMS[baseline_start_i:baseline_end_i])

    # Too little samples to fit parameters to --> Add additional datapoints
    if end_i - start_i < 3:
        plus_idx = 3 - (end_i - start_i)
    else:
        plus_idx = 0
    
    # Plot the fitted bell-curve relative to the sEAdi peak
    # sEAdi peak, on- and off-set
    axis[2].plot(t_emg[sEAdi_peaks[idx]],
                      y_di_RMS[sEAdi_peaks[idx]], '*r')
    axis[2].plot(t_emg[start_i], y_di_RMS[start_i], '*r')
    axis[2].plot(t_emg[end_i], y_di_RMS[end_i], '*r')
    try:
        popt, pcov = curve_fit(func, 
                        t_emg[start_i:end_i+1], 
                        y_di_RMS[start_i:end_i+1]-y_di_min[idx],
                        bounds=([-np.inf, t_emg[sEAdi_peaks[idx]]-0.5, -np.inf], 
                                [np.inf, t_emg[sEAdi_peaks[idx]]+0.5, np.inf])
                        )

        # Fitted bell-curve in green
        axis[2].plot(t_emg[start_plot_i:end_plot_i+1], 
                        func(t_emg[start_plot_i:end_plot_i+1], *popt)
                        + y_di_min[idx], 'g')
    except(RuntimeError):
        pass

axis[2].set_ylabel('sEAdi (uV)')
if len(sEAdi_peaks) > 0:
    axis[2].set_ylim([0, 1.4*np.percentile(y_di_RMS[sEAdi_peaks], 90)])

# sEApara:
axis[3].grid(True)
N_samp = len(sEAdi[int(start_s):int(end_s)])
#   sEApara signal
axis[3].plot([y / fs_emg for y in range(N_samp)],
             sEApara[int(start_s):int(end_s)])
#   Rolling baseline
axis[3].plot([y / fs_emg for y in range(N_samp)],
             sEApara_rolling_baseline[:int(end_s)-int(start_s)])
# axis[3].plot([y / fs_emg for y in range(N_samp)],
#              sEApara_rolling_baseline_aug[:int(end_s)-int(start_s)])

# Plot bell-curve error of sEApara peaks
for idx in range(len(sEApara_peaks)):
    start_i = sEApara_starts[idx]
    end_i = sEApara_ends[idx]

    start_plot_i = max([0, start_i-fs_emg//4])
    end_plot_i = min([N_samp, end_i+fs_emg//4])
    
    baseline_start_i = max([0, sEApara_peaks[idx] - 5*fs_emg])
    baseline_end_i = min(
        [len(y_para_RMS) - 1, sEApara_peaks[idx] + 5*fs_emg])
    y_para_min[idx] = min(y_para_RMS[baseline_start_i:baseline_end_i])

    # Too little samples to fit parameters to --> Add additional datapoints
    if end_i - start_i < 3:
        plus_idx = 3 - (end_i - start_i)
    else:
        plus_idx = 0
    
    # Plot the fitted bell-curve relative to the sEApara peak
    # sEApara peak, on- and off-set
    axis[3].plot(t_emg[sEApara_peaks[idx]],
                      y_para_RMS[sEApara_peaks[idx]], '*r')
    axis[3].plot(t_emg[start_i], y_para_RMS[start_i], '*r')
    axis[3].plot(t_emg[end_i], y_para_RMS[end_i], '*r')
    try:
        popt, pcov = curve_fit(func, 
                        t_emg[start_i:end_i+1], 
                        y_para_RMS[start_i:end_i+1]-y_para_min[idx],
                        bounds=([-np.inf, t_emg[sEApara_peaks[idx]]-0.5, -np.inf], 
                                [np.inf, t_emg[sEApara_peaks[idx]]+0.5, np.inf])
                        )

        # Fitted bell-curve in green
        axis[3].plot(t_emg[start_plot_i:end_plot_i+1], 
                        func(t_emg[start_plot_i:end_plot_i+1], *popt)
                        + y_para_min[idx], 'g')
    except(RuntimeError):
        pass

axis[3].set_ylabel('sEApara (uV)')
axis[3].set_xlabel('t (s)')
if len(sEApara_peaks) > 0:
    axis[3].set_ylim([0, 1.4*np.percentile(y_para_RMS[sEApara_peaks], 90)])

## 11. Add the outcome data to a dataframe

In [ ]:
print(patient + ' / ' + measurement_date + ' / ' + PS_step_chosen)

In [ ]:
fig_1.set_figwidth(fig_w)
fig_1.savefig(main_output_dir + '/' + patient + '/' + measurement_date
              + '/' + measurement_date + '_' + patient + '_' 
              + PS_step_chosen + '_timeplots_'+str(int(gate_width*1000))+'.png', 
              dpi=300)
fig_2.set_figwidth(fig_w)
fig_2.savefig(main_output_dir + '/' + patient + '/' + measurement_date
              + '/' + measurement_date + '_' + patient + '_' 
              + PS_step_chosen + '_AUB_errors_'+str(int(gate_width*1000))+'.png', 
              dpi=300)
fig_3.set_figwidth(fig_w)
fig_3.savefig(main_output_dir + '/' + patient + '/' + measurement_date
              + '/' + measurement_date + '_' + patient + '_' 
              + PS_step_chosen + '_bell_errors_'+str(int(gate_width*1000))+'.png', 
              dpi=300)

In [ ]:
# di_data_list = []
# para_data_list = []
# Paw_data_list = []
# Vvent_data_list = []



# # PS_step_del = '002'
# PS_step_del = PS_step_chosen
# df_di = df_di.drop(df_di[df_di['measurement'] == PS_step_del].index)
# df_para = df_para.drop(df_para[df_para['measurement'] == PS_step_del].index)
# df_Paw = df_Paw.drop(df_Paw[df_Paw['measurement'] == PS_step_del].index)
# df_Vvent = df_Vvent.drop(df_Vvent[df_Vvent['measurement'] == PS_step_del].index)

# di_data_list = list(df_di.values)
# para_data_list = list(df_para.values)
# Paw_data_list = list(df_Paw.values)
# Vvent_data_list = list(df_Vvent.values)

# df_di

In [ ]:
# Store sEAdi parameters in a dataframe (~ table)
columns_di = ['patient', 'measurement', 'PEEP_set', 'PS_set', 
              'ETP_di', 'SNR_di', 'ETP_di_baseline', 'ETP_di_bell_SE',
              't_delta_di_med', 't_delta_ecg_med', 
              't_di_peak', 't_di_start', 't_di_end', 
              'P_peak_associated', 'V_peak_associated', 
              'baseline_di_mean', 'baseline_di_std']

df_di = pd.DataFrame(di_data_list, columns=columns_di)

if len(df_di[df_di['measurement'] == PS_step_chosen]) == 0:
    for idx in range(len(sEAdi_peaks)):
        data_di_tmp = [patient, PS_step_chosen, PEEP_set, PS_set, 
                       ETP_di_occs[idx], SNR_di[idx], 
                       ETP_di_baseline[idx], ETP_di_bell_error[idx], 
                       t_delta_di_med, t_delta_ecg_med,
                       sEAdi_peaks[idx]/fs_emg, 
                       sEAdi_starts[idx]/fs_emg,
                       sEAdi_ends[idx]/fs_emg,
                       sEAdi_links[idx, 0]/fs_emg,
                       sEAdi_links[idx, 1]/fs_emg,
                       sEAdi_bl_mean,
                       sEAdi_bl_std
                    ]
        
        di_data_list.append(data_di_tmp)
    
    df_di = pd.DataFrame(di_data_list, columns=columns_di)
else:
    print('Data of PS step ' + PS_step_chosen + ' already added to the ' 
          + 'dataframe. Don''t add the data twice!')
    
# df_di

In [ ]:
# Store sEApara parameters in a dataframe (~ table)
columns_para = ['patient', 'measurement', 'PEEP_set', 'PS_set', 
              'ETP_para', 'SNR_para', 'ETP_para_baseline', 'ETP_para_bell_SE',
              't_delta_para_med', 't_delta_ecg_med', 
              't_para_peak', 't_para_start', 't_para_end', 
              'Paw_associated', 'Vvent_associated', 
              'baseline_para_mean', 'baseline_para_std']

df_para = pd.DataFrame(para_data_list, columns=columns_para)

if len(df_para[df_para['measurement'] == PS_step_chosen]) == 0:
    for idx in range(len(sEApara_peaks)):
        data_para_tmp = [patient, PS_step_chosen, PEEP_set, PS_set, 
                       ETP_para_occs[idx], SNR_para[idx], 
                       ETP_para_baseline[idx], ETP_para_bell_error[idx], 
                       t_delta_para_med, t_delta_ecg_med,
                       sEApara_peaks[idx]/fs_emg, 
                       sEApara_starts[idx]/fs_emg,
                       sEApara_ends[idx]/fs_emg,
                       sEApara_links[idx, 0]/fs_emg,
                       sEApara_links[idx, 1]/fs_emg,
                       sEApara_bl_mean,
                       sEApara_bl_std
                    ]
        
        para_data_list.append(data_para_tmp)
    
    df_para = pd.DataFrame(para_data_list, columns=columns_para)
else:
    print('Data of PS step ' + PS_step_chosen + ' already added to the ' 
          + 'dataframe. Don''t add the data twice!')
    
# df_para

In [ ]:
# Store the Paw output parameters in a dataframe (~ table)
columns_Paw = ['patient', 'measurement', 'PEEP_set', 'PS_set', 
                   'Paw_peak', 't_P_peak', 
                   'sEAdi_associated', 'sEApara_associated'
               ]

df_Paw = pd.DataFrame(Paw_data_list, columns=columns_Paw)

if len(df_Paw[df_Paw['measurement'] == PS_step_chosen]) == 0:
    for idx in range(len(Paw_PKS)):
        Paw_tmp = [patient, PS_step_chosen, PEEP_set, PS_set,
                   Paw_sel[Paw_PKS[idx]],
                   Paw_PKS[idx]/fs_vent, 
                   Paw_links[idx, 0]/fs_emg,
                   Paw_links[idx, 1]/fs_emg
                   ]
        
        Paw_data_list.append(Paw_tmp)
    
    df_Paw = pd.DataFrame(Paw_data_list, columns=columns_Paw)
else:
    print('Data of PS step ' + PS_step_chosen + ' already added to the ' 
          + 'dataframe. Don''t add the data twice!')
    
# df_Paw

In [ ]:
# Store the Vvent output parameters in a dataframe (~ table)

columns_Vvent = ['patient', 'measurement', 'PEEP_set', 'PS_set', 
                   'TV', 't_TV_peak', 
                   'sEAdi_associated', 'sEApara_associated'
               ]

df_Vvent = pd.DataFrame(Vvent_data_list, columns=columns_Vvent)

if len(df_Vvent[df_Vvent['measurement'] == PS_step_chosen]) == 0:
    for idx in range(len(V_ei_PKS)):
        Vvent_tmp = [patient, PS_step_chosen, PEEP_set, PS_set,
                     V_vent[V_ei_PKS[idx]],
                     V_ei_PKS[idx]/fs_vent, 
                     Vvent_links[idx, 0]/fs_emg,
                     Vvent_links[idx, 1]/fs_emg
                    ]
        
        Vvent_data_list.append(Vvent_tmp)
    
    df_Vvent = pd.DataFrame(Vvent_data_list, columns=columns_Vvent)
else:
    print('Data of PS step ' + PS_step_chosen + ' already added to the ' 
          + 'dataframe. Don''t add the data twice!')
    
# df_Vvent

In [ ]:
# Generate read-me text file describing the used settings during analysis
if gate_twice == True:
    N_gated = 2
else:
    N_gated = 1

context = (patient + ' / ' + measurement_date + ' / ' + PS_step_chosen + '\n'
        +'t_start: ' + str(start) + ' s' + '\n'
        +'t_end: ' + str(end) + ' s' + '\n'
        +'gate_width: ' + str(int(gate_width*1000)) + ' ms' + '\n'
        +'gate_threshold: ' + str(peak_fraction) + '' + '\n'
        +'gate_ECG_shift: ' + str(ECG_shift) + ' samples' + '\n'
        +'time_shift: ' + str(time_shift_default) + ' s' + '\n'
        +'TV_percentile: ' + str(TV_percentile) + '\n'
        +'TV_fraction: ' + str(TV_fraction) + '\n'
        +'sEAdi_prominence_factor: ' + str(sEAdi_prominence_factor) + '\n'
        +'sEApara_prominence_factor: ' + str(sEApara_prominence_factor) + '\n'
        +'emg_peak_distance: ' + str(emg_peak_distance) + ' s\n'
        +'PEEP_set: ' + str(PEEP_set) + '' + '\n' \
        +'PS_set: ' + str(PS_set) + '' + '\n' \
        +'N_gated: ' + str(N_gated) + '' + '\n' \
        +'augmented_percentile: ' + str(augmented_perc) + ' %' + '\n'
        +'analysis_date: ' + str(datetime.now()) + ''
        )

with open(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient + '_' + PS_step_chosen
              + '_readme.txt', 'w') as f:
    f.write(context)
    
print(context)

## 12. Plot the sEAdi versus PS

In [ ]:
# Move on to the next measurement
if PS_step_chosen in df_Vvent['measurement'].to_list():
    print('Data of measurement ' + PS_step_chosen + ' added to the '
            + 'dataframe. PS step ' + str(PS_step_chosen) + ' (' +
            str(len(set(df_Vvent['measurement'].values))) + '/1) is '
            + 'successfully evaluated and stored. You can continue to the '
            + 'saving the overall data!')
else:
    print('Data of measurement ' + PS_step_chosen + ' not yet added to the ' 
        + 'dataframe!')

['To next PS step'](#section_ps_step)

In [ ]:
# Store session data if complete
if ((len(set(df_Vvent['measurement'].values)) < 1) |
    (measurement_date != measurement_dates[date_idx])):
    print('Warning: Not 1 PS settings evaluated yet!')
else:
    df_di.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient
              +'_sEAdi.csv')
    df_para.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient
              +'_sEApara.csv')
    df_Paw.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient
              +'_Paw.csv')
    df_Vvent.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient
              +'_Vvent.csv')

    # fig_4.savefig(main_output_dir + '/' + patient + '/' 
    #                   + measurement_date  + '/' + measurement_date 
    #                   + '_' + patient + '_ETPs_scatter.png', dpi=300)

    
    print('Notification: Data of 1 PS settings stored!')
    print(measurement_date)

In [ ]:
# Move on to the next measurement date if data saved

if not os.path.exists(main_output_dir + '/' + patient + '/' + measurement_date  
                      + '/' + measurement_date + '_' + patient +'_Vvent.csv'):
    print('Warning 1: Data not stored! Do not progress to next date!')
elif ((len(set(df_Vvent['measurement'].values)) < 1)):
    print('Warning: Data not stored! Do not progress to next date!')
elif ((date_idx < len(measurement_dates)
       and (measurement_date != measurement_dates[date_idx]))):
    print('Notification: Data appropriately stored and date index increased.'
              +'You may progress to next date:')
    print(measurement_dates[date_idx] + ' (day ' + str(date_idx+1) + '/'
          + str(len(measurement_dates)) + ')')
else:
    if date_idx < len(measurement_dates)-1:
        print('Notification: Data appropriately stored. '
              +'You may progress to next date:')
        
        if measurement_date == measurement_dates[date_idx]:
            ps12_it += 1
            date_idx += 1            
            print(measurement_dates[date_idx] + ' (day ' + str(date_idx+1) + '/'
                  + str(len(measurement_dates)) + ')')
    else:
        if patient == patients[patient_idx]:
            ps12_it += 1
            patient_idx += 1
            date_idx = 0
        print('Notification: Data appropriately stored. \n'+
              'This was the last measurement of this patient. '
              +'You may progress to next patient!')

In [ ]:
print('Notification: Finished dataset ' + str(ps12_it) + '/' + str(len(ps12_df))
      + ' (' + str(np.round(ps12_it/len(ps12_df)*100, 2)) + '%)')

In [ ]:
ps12_it

In [ ]:
# # Save data anyway:
# df_di.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
#             + '/' + measurement_date + '_' + patient
#             +'_sEAdi.csv')
# df_para.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
#             + '/' + measurement_date + '_' + patient
#             +'_sEApara.csv')
# df_Paw.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
#             + '/' + measurement_date + '_' + patient
#             +'_Paw.csv')
# df_Vvent.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
#             + '/' + measurement_date + '_' + patient
#             +'_Vvent.csv')

# fig_4.savefig(main_output_dir + '/' + patient + '/' 
#                     + measurement_date  + '/' + measurement_date 
#                     + '_' + patient + '_ETPs_scatter.png', dpi=300)

# # bp_di.figure.savefig(main_output_dir + '/' + patient + '/' 
# #                     + measurement_date  + '/' + measurement_date 
# #                     + '_' + patient + '_sEAdi_scatter.png', dpi=300)


# # bp_para.figure.savefig(main_output_dir + '/' + patient + '/' 
# #                   + measurement_date  + '/' + measurement_date 
# #                   + '_' + patient + '_sEApara_scatter.png', dpi=300)

# date_idx += 1
# if date_idx < len(measurement_dates):
#     print('Notification: Data appropriately stored. '
#             +'You may progress to next date!')
#     print(measurement_date)
# else:
#     if patient == patients[patient_idx]:
#         patient_idx += 1
#         date_idx = 0
#     print('Notification: Data appropriately stored. \n'+
#             'This was the last measurement of this patient. '
#             +'You may progress to next patient!')


In [ ]:
# # Move on to the next measurement date
# date_idx += 1

In [ ]:
# # Move on to the previous measurement date
# date_idx -= 1

In [ ]:
# date_idx

['To next measurement date'](#section_date_step)

In [ ]:
# Move on to the next patient
# patient_idx += 1

In [ ]:
# # Move on to the previous patient
# patient_idx -= 1

['To next patient'](#section_patient_step)

In [ ]:
# gate_twice = True